In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB



################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from pandas import read_csv
from searchUtils import findPatternExt

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)



# Parse Spotify

In [2]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
from spotifyCharts import spotifyCharts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
sd = spotifyData(minYear=1, maxYear=2021)
sd.parse()
#sd.saveChartData()

In [ ]:
sc = spotifyCharts()
sc.getChartsByRank(2)

In [ ]:
# Full 

In [ ]:
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)

In [ ]:
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[0,1])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()

# Download New Spotify Data

In [3]:
def downloadURL(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    print("Now Downloading {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need
        
    return data, response.getcode()

# Global Parameters

In [2]:
from datetime import datetime

# Regional Data (Top 2000)

In [3]:
countries = ['sa', 'ie', 'ar', 'no', 'ma', 'in', 'cl', 'hu', 'ch', 'ru', 'lu', 'pt', 'bg', 'jp', 'hn', 'se', 'vn', 'ro', 'hk', 'ec', 'cr', 'nz', 
             'id', 'gb', 'fr', 'sk', 'pe', 'pa', 'lt', 'br', 'ua', 'gr', 'it', 'es', 'fi', 'eg', 'cz', 'uy', 'il', 'py', 'do', 'is', 'at', 'dk',
             'sg', 'ni', 'ca', 'be', 'mx', 'tr', 'de', 'co', 'tw', 'us', 'ae', 'gt', 'pl', 'sv', 'th', 'nl', 'ee', 'ph', 'bo', 'au', 'my', 'za', 'lv']

from pandas import date_range
from datetime import datetime
start = '12/23/2016'
start = '12/1/2020'

end   = datetime.now().strftime("%Y-%m-%d")
#end   = "2021-03-20"
datevals = [x.strftime('%Y-%m-%d') for x in list(date_range(start=start, end=end, freq='W-FRI'))]
datevals.reverse()
print(datevals)

['2021-05-28', '2021-05-21', '2021-05-14', '2021-05-07', '2021-04-30', '2021-04-23', '2021-04-16', '2021-04-09', '2021-04-02', '2021-03-26', '2021-03-19', '2021-03-12', '2021-03-05', '2021-02-26', '2021-02-19', '2021-02-12', '2021-02-05', '2021-01-29', '2021-01-22', '2021-01-15', '2021-01-08', '2021-01-01', '2020-12-25', '2020-12-18', '2020-12-11', '2020-12-04']


In [4]:
from time import sleep
import urllib
from fsUtils import isFile
from ioUtils import saveFile

for i,country in enumerate(countries):    
    print("\n\n")
    print("==========>",country,"<=========")
    nDownload = 0
    errs = 0
    for i,dateval in enumerate(datevals):
        if errs > 10:
            break
        if nDownload > 3000:
            break
        if i == len(datevals)-1:
            break
        start = datevals[i+1]
        end   = datevals[i]
        url      = "https://spotifycharts.com/regional/{0}/weekly/{1}--{2}/download".format(country, start, end)
        #savename = "/Users/tgadfort/Documents/code/charts/spotify/
        savename = "/Volumes/Piggy/Charts/data/spotify/categories/regional-{0}-weekly-{1}--{2}.p".format(country, start, end)
        data = None
        
        if isFile(savename):
            continue

        if True:
            try:
                data, code = downloadURL(url)
            except:
                errs += 1
                continue
            
        saveFile(idata=data, ifile=savename)
        sleep(3.5)
        nDownload += 1




==========> sa <=========



==========> ie <=========



==========> ar <=========



==========> no <=========



==========> ma <=========



==========> in <=========



==========> cl <=========



==========> hu <=========



==========> ch <=========



==========> ru <=========



==========> lu <=========



==========> pt <=========



==========> bg <=========



==========> jp <=========



==========> hn <=========



==========> se <=========



==========> vn <=========



==========> ro <=========



==========> hk <=========



==========> ec <=========



==========> cr <=========



==========> nz <=========



==========> id <=========



==========> gb <=========



==========> fr <=========



==========> sk <=========



==========> pe <=========



==========> pa <=========



==========> lt <=========



==========> br <=========



==========> ua <=========



==========> gr <=========



==========> it <=========



==========> es <=========



==========>

In [3]:
from spotifyData import spotifyData
sp = spotifyData()
sp.parse()

==> sa
/Volumes/Piggy/Charts/data/spotify/categories
Found 123 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-sa-weekly.p
  --> This file is 301.3kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-sa-weekly.p
  --> This file is 301.3kB.
==> ie
/Volumes/Piggy/Charts/data/spotify/categories
Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-ie-weekly.p
  --> This file is 614.4kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-ie-weekly.p
  --> This file is 614.4kB.
==> ar
/Volumes/Piggy/Charts/data/spotify/categories
Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-ar-weekly.p
  --> This file is 575.5kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-ar-weekly.p
  --> This file is 575.5kB.
==> no
/Volumes/Piggy/Charts/data/spotify/categories
Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-no-weekly.p
  

Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-br-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-br-weekly.p
  --> This file is 1.2MB.
==> ua
/Volumes/Piggy/Charts/data/spotify/categories
Found 34 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-ua-weekly.p
  --> This file is 177.6kB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-ua-weekly.p
  --> This file is 177.6kB.
==> gr
/Volumes/Piggy/Charts/data/spotify/categories
Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-gr-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-gr-weekly.p
  --> This file is 1.2MB.
==> it
/Volumes/Piggy/Charts/data/spotify/categories
Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-it-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/result

Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-nl-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-nl-weekly.p
  --> This file is 1.2MB.
==> ee
/Volumes/Piggy/Charts/data/spotify/categories
Found 217 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-ee-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-ee-weekly.p
  --> This file is 1.2MB.
==> ph
/Volumes/Piggy/Charts/data/spotify/categories
Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-ph-weekly.p
  --> This file is 1.2MB.
Saved data to /Volumes/Piggy/Charts/data/spotify/results/regional-ph-weekly.p
  --> This file is 1.2MB.
==> bo
/Volumes/Piggy/Charts/data/spotify/categories
Found 218 files.
Saving data to /Volumes/Piggy/Charts/data/spotify/results/regional-bo-weekly.p
  --> This file is 1.1MB.
Saved data to /Volumes/Piggy/Charts/data/

# Viral Data (Top 50)

In [11]:
countries = ['sa', 'ie', 'ar', 'no', 'ma', 'in', 'cl', 'hu', 'ch', 'ru', 'lu', 'pt', 'bg', 'jp', 'hn', 'se', 'vn', 'ro', 'hk', 'ec', 'cr', 'nz', 
             'id', 'gb', 'fr', 'sk', 'pe', 'pa', 'lt', 'br', 'ua', 'gr', 'it', 'es', 'fi', 'eg', 'cz', 'uy', 'il', 'py', 'do', 'is', 'at', 'dk',
             'sg', 'ni', 'ca', 'be', 'mx', 'tr', 'de', 'co', 'tw', 'us', 'ae', 'gt', 'pl', 'sv', 'th', 'nl', 'ee', 'ph', 'bo', 'au', 'my', 'za', 'lv']

from pandas import date_range
from datetime import datetime
start = '12/23/2016'
start = '12/1/2020'

end   = datetime.now().strftime("%Y-%m-%d")
end   = "2021-03-31"
datevals = [x.strftime('%Y-%m-%d') for x in list(date_range(start=start, end=end, freq='W-THU'))]
datevals.reverse()
print(datevals)

['2021-03-25', '2021-03-18', '2021-03-11', '2021-03-04', '2021-02-25', '2021-02-18', '2021-02-11', '2021-02-04', '2021-01-28', '2021-01-21', '2021-01-14', '2021-01-07', '2020-12-31', '2020-12-24', '2020-12-17', '2020-12-10', '2020-12-03']


In [12]:
from time import sleep
import urllib
from fsUtils import isFile
from ioUtils import saveFile

for country in countries:
    print("\n\n")
    print("==========>",country,"<=========")
    nDownload = 0
    errs = 0
    for i,dateval in enumerate(datevals):
        if errs > 10:
            break
        if nDownload > 3000:
            break
        if i == len(datevals)-1:
            break
        start = datevals[i+1]
        end   = datevals[i]
        url      = "https://spotifycharts.com/viral/{0}/weekly/{1}--{2}/download".format(country, start, start)
        #2021-01-14--2021-01-14
        #url      = "https://spotifycharts.com/regional/{0}/weekly/{1}--{2}/download".format(country, start, end)
        #savename = "/Users/tgadfort/Documents/code/charts/spotify/
        savename = "/Volumes/Piggy/Charts/data/spotifyviral/categories/viral-{0}-weekly-{1}--{2}.p".format(country, start, start)
        if isFile(savename):
            continue

        try:
            data, code = downloadURL(url)
        except:
            errs += 1
            sleep(2)
            continue
            
        saveFile(idata=data, ifile=savename)
        sleep(3.5)
        nDownload += 1




==========> sa <=========
Now Downloading https://spotifycharts.com/viral/sa/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/sa/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/sa/weekly/2021-03-04--2021-03-04/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/sa/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.2kB.



==========> ie <=========
Now Downloading https://spotifycharts.com/viral/ie/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.4kB.
Now Downloading https://spotifycharts.com/viral/ie/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ie/weekly/2021-03-04--2021-03-04/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ie/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.3kB.



==========>

  --> This file is 2.5kB.
Now Downloading https://spotifycharts.com/viral/vn/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.5kB.



==========> ro <=========
Now Downloading https://spotifycharts.com/viral/ro/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/ro/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ro/weekly/2021-03-04--2021-03-04/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ro/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.3kB.



==========> hk <=========
Now Downloading https://spotifycharts.com/viral/hk/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.6kB.
Now Downloading https://spotifycharts.com/viral/hk/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.5kB.
Now Downloading https://spotifycharts.com/viral/hk/weekly/2021-03-04--2021-03-04/download
  --> This fil

  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/es/weekly/2021-03-04--2021-03-04/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/es/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.2kB.



==========> fi <=========
Now Downloading https://spotifycharts.com/viral/fi/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/fi/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/fi/weekly/2021-03-04--2021-03-04/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/fi/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.2kB.



==========> eg <=========
Now Downloading https://spotifycharts.com/viral/eg/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.4kB.
Now Downloading https://spotifycharts.com/viral/eg/weekly/2021-03-11--2021-03-11/download
  --> This fil

Now Downloading https://spotifycharts.com/viral/be/weekly/2021-01-28--2021-01-28/download
  --> This file is 2.1kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2021-01-21--2021-01-21/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2021-01-14--2021-01-14/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2021-01-07--2021-01-07/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2020-12-31--2020-12-31/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2020-12-24--2020-12-24/download
  --> This file is 2.4kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2020-12-17--2020-12-17/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2020-12-10--2020-12-10/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/be/weekly/2020-12-03--20

  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-03-04--2021-03-04/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-02-25--2021-02-25/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-01-28--2021-01-28/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-01-21--2021-01-21/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-01-14--2021-01-14/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2021-01-07--2021-01-07/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ae/weekly/2020-12-31--2020-12-31/download
  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/vira

  --> This file is 2.2kB.
Now Downloading https://spotifycharts.com/viral/nl/weekly/2021-01-07--2021-01-07/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/nl/weekly/2020-12-31--2020-12-31/download
  --> This file is 2.4kB.
Now Downloading https://spotifycharts.com/viral/nl/weekly/2020-12-24--2020-12-24/download
  --> This file is 2.4kB.
Now Downloading https://spotifycharts.com/viral/nl/weekly/2020-12-17--2020-12-17/download
  --> This file is 2.5kB.
Now Downloading https://spotifycharts.com/viral/nl/weekly/2020-12-10--2020-12-10/download
  --> This file is 2.4kB.
Now Downloading https://spotifycharts.com/viral/nl/weekly/2020-12-03--2020-12-03/download
  --> This file is 2.3kB.



==========> ee <=========
Now Downloading https://spotifycharts.com/viral/ee/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.3kB.
Now Downloading https://spotifycharts.com/viral/ee/weekly/2021-03-11--2021-03-11/download
  --> This file is 2.3kB.
Now Downloading h

Now Downloading https://spotifycharts.com/viral/za/weekly/2020-12-10--2020-12-10/download



==========> lv <=========
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-03-18--2021-03-18/download
  --> This file is 2.5kB.
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-03-11--2021-03-11/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-03-04--2021-03-04/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-02-25--2021-02-25/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-01-28--2021-01-28/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-01-21--2021-01-21/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-01-14--2021-01-14/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2021-01-07--2021-01-07/download
Now Downloading https://spotifycharts.com/viral/lv/weekly/2020-12-31--2020-12-31/download
Now Downloading https://spotifycharts.com/vir

In [4]:
from spotifyViral import spotifyViralData
sp = spotifyViralData()
sp.parse()

==> sa
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 122 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-sa-weekly.p
  --> This file is 210.6kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-sa-weekly.p
  --> This file is 210.6kB.
==> ie
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ie-weekly.p
  --> This file is 436.0kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ie-weekly.p
  --> This file is 436.0kB.
==> ar
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ar-weekly.p
  --> This file is 405.9kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ar-weekly.p
  --> This file is 405.9kB.
==> no
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotify

Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-lt-weekly.p
  --> This file is 408.4kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-lt-weekly.p
  --> This file is 408.4kB.
==> br
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-br-weekly.p
  --> This file is 416.5kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-br-weekly.p
  --> This file is 416.5kB.
==> ua
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 36 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ua-weekly.p
  --> This file is 66.0kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ua-weekly.p
  --> This file is 66.0kB.
==> gr
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 220 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-gr-weekly.p
  --> This file is 399.3kB.
S

Found 16 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-sv-weekly.p
  --> This file is 28.3kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-sv-weekly.p
  --> This file is 28.3kB.
==> th
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 16 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-th-weekly.p
  --> This file is 28.8kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-th-weekly.p
  --> This file is 28.8kB.
==> nl
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 16 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-nl-weekly.p
  --> This file is 30.3kB.
Saved data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-nl-weekly.p
  --> This file is 30.3kB.
==> ee
/Volumes/Piggy/Charts/data/spotifyviral/categories
Found 16 files.
Saving data to /Volumes/Piggy/Charts/data/spotifyviral/results/viral-ee-weekly.p
  --> This file is 29.4kB.
Saved dat

# Extras

In [ ]:
countries = ['sa', 'ie', 'ar', 'no', 'ma', 'in', 'cl', 'hu', 'ch', 'ru', 'lu', 'pt', 'bg', 'jp', 'hn', 'se', 'vn',
             'ro','hk', 'ec', 'cr', 'nz', 'id', 'gb', 'fr', 'sk', 'pe', 'pa', 'lt', 'br', 'ua', 'gr', 'it', 'es',
             'fi', 'eg', 'cz', 'uy', 'il', 'py', 'do', 'is', 'at', 'dk', 'sg', 'ni', 'ca', 'be', 'mx', 'tr', 'de',
             'co', 'tw', 'us', 'ae', 'gt', 'pl', 'sv', 'th', 'nl', 'ee', 'ph', 'bo', 'au', 'my', 'za', 'lv']
names = ["Saudi Arabia", "Ireland", "Argentina", "Norway", "India", "Morocco", "Chile", "Hungary", "Switzerland", "Russia", "Luxembourg", "Portugal", "Bulgaria", "Japan", "Honduras", "Sweden", "Vietnam",
        "Romania", "Hong Kong", "Ecuador", "Costa Rica", "New Zealand", "Indonesia", "United Kingdom", "France", "Slovakia", "Peru", "Panama", "Lithuania", "Brazil", "Ukraine", "Greece", "Italy", "Spain",
        "Finland", "Egypt", "Czech Republic", "Uraguay", "Isreal", "Paraguay", "Dominican Republic", "Iceland", "Austria", "Denmark", "Singapore", "Nicaragua", "Canada", "Belgium", "Mexico", "Turkey", "Germany",
        "Colombia", "Taiwan", "United States", "United Arab Emirates", "Guatemala", "Poland", "El Salvador", "Thailand", "Netherlands", "Estonia", "Philippines", "Bolivia", "Australia", "Malaysia", "South Africa", "Latvia"]
comb = dict(zip(countries, names))

In [ ]:
print(comb)

In [ ]:
for abbr,name in comb.items():
    print("        self.{0: <19} = ['regional-{1}-weekly']".format(name.replace(" ", "").lower(), abbr))

In [ ]:
chartRanks = {}
chartRanks[0] = ['unitedstates', 'canada']
chartRanks[1] = ['unitedkingdom', 'australia', 'ireland', 'newzealand']
chartRanks[2] = ['germany', 'austria', 'switzerland']
chartRanks[3] = ['italy', 'france', 'spain', 'netherlands', 'luxembourg']
chartRanks[4] = ['finland', 'iceland', 'belgium', 'czechrepublic', 'hungary']
chartRanks[5] = ['denmark', 'sweden', 'norway']
chartRanks[6] = ['isreal', 'turkey', 'saudiarabia', 'unitedarabemirates', 'egypt', 'morocco']
chartRanks[7] = ['russia', 'ukraine', 'bulgaria', 'latvia', 'estonia', 'lithuania', 'slovakia', 'romania', 'poland']
chartRanks[8] = ['mexico', 'costarica', 'panama', 'dominicanrepublic', 'guatemala', 'nicaragua', 'honduras', 'elsalvador']        
chartRanks[9] = ['colombia', 'argentina', 'peru', 'paraguay', 'uraguay', 'chile', 'bolivia', 'ecuador']
chartRanks[10] = ['japan', 'hongkong', 'vietnam', 'taiwan', 'thailand', 'malaysia', 'singapore']
chartRanks[11] = ['philippines', 'greece', 'indonesia', 'india', 'southafrica']
chartRanks[12] = ['brazil', 'portugal']
chartRanks[13] = ['global']

In [ ]:
from listUtils import getFlatList
ranks = getFlatList(chartRanks.values())

In [ ]:
countriesList = set([name.replace(" ", "").lower() for name in comb.values()])
ranksList = set(ranks)
countriesList.symmetric_difference(ranksList)

In [ ]:
len(countriesList)

In [ ]:
len(ranksList)

In [ ]:
ranksList.difference(countriesList)